In [1]:
import sqlite3
import pandas as pd
import time
import requests

conexion = sqlite3.connect("NBA_DATA.db")

cursor = conexion.cursor()

# Crear la tabla Teams
cursor.execute("""
create table If Not Exists Teams (
  team_id int primary key,
  team_nickname text not null,
  abbreviation text not null,
  city text not null,
  state text not null,
  full_name text not null
)
""")

# Crear la tabla Games
cursor.execute("""
create table If Not Exists Games (
  GAME_ID bigint primary key,
  game_date date not null,
  h_team_nickname text not null,
  a_team_nickname text not null,
  h_team_id int references teams (team_id),
  a_team_id int references teams (team_id),
  season int not null
)
""")

# Crear la tabla TEAM_INFO_COMMON
cursor.execute("""
create table If Not Exists game_stats (
  GAME_ID bigint references games (GAME_ID),
  HOME_FLAG boolean not null,
  AWAY_FLAG boolean not null,
  CITY text,
  NICKNAME text,
  team_id int,
  W int,
  L int,
  W_HOME int,
  L_HOME int,
  W_ROAD int,
  L_ROAD int,
  TEAM_TURNOVERS int,
  TEAM_REBOUNDS int,
  GP int,
  GS int,
  ACTUAL_MINUTES int,
  ACTUAL_SECONDS int,
  FG int,
  FGA int,
  FG_PCT double precision,
  FG3 int,
  FG3A int,
  FG3_PCT double precision,
  FT int,
  FTA int,
  FT_PCT double precision,
  OFF_REB int,
  DEF_REB int,
  TOT_REB int,
  AST int,
  PF int,
  STL int,
  TOTAL_TURNOVERS int,
  BLK int,
  PTS int,
  AVG_REB double precision,
  AVG_PTS double precision,
  DQ int,
  OFFENSIVE_EFFICIENCY double precision,
  SCORING_MARGIN double precision,
  HOME_WIN_PCTG double precision,
  AWAY_WIN_PCTG double precision,
  TOTAL_WIN_PCTG double precision,
  ROLLING_SCORING_MARGIN int,
  ROLLING_OE double precision,
  NUM_REST_DAYS int,
  primary key (GAME_ID, HOME_FLAG)
)
""")

conexion.close()

In [2]:
from nba_api.stats.static import teams

teamLookup = pd.DataFrame(teams.get_teams())

teamLookup = teamLookup.rename(columns={"id": "team_id", "nickname": "team_nickname"}).drop(columns='year_founded')

conexion = sqlite3.connect("NBA_DATA.db")

teamLookup.to_sql("Teams", conexion, if_exists="replace", index=False)

30

In [3]:
def retry(func, retries=3):
    def retry_wrapper(*args, **kwargs):
        attempts = 0
        while attempts < retries:
            try:
                return func(*args, **kwargs)
            except requests.exceptions.RequestException as e:
                print(e)
                time.sleep(30)
                attempts += 1

    return retry_wrapper

In [4]:
from nba_api.stats.static import teams, players
from nba_api.stats.endpoints import cumestatsteamgames, cumestatsteam, gamerotation
import pandas as pd
import numpy as np
import json
import difflib
import time
import requests
import sqlite3

def getSeasonScheduleFrame(seasons, seasonType):
    def getGameDate(matchup):
        return matchup.partition(' at')[0][:10]

    def getHomeTeam(matchup):
        return matchup.partition(' at')[2]

    def getAwayTeam(matchup):
        return matchup.partition(' at')[0][10:]

    def getTeamIDFromNickname(nickname):
        return teamLookup.loc[teamLookup['team_nickname'] == difflib.get_close_matches(nickname, teamLookup['team_nickname'], 1)[0]].values[0][0]

    @retry
    def getRegularSeasonSchedule(season, teamID, seasonType):
        season = str(season) + "-" + str(season + 1)[-2:]
        teamGames = cumestatsteamgames.CumeStatsTeamGames(league_id='00', season=season,
                                                          season_type_all_star=seasonType,
                                                          team_id=teamID).get_normalized_json()

        teamGames = pd.DataFrame(json.loads(teamGames)['CumeStatsTeamGames'])
        teamGames['SEASON'] = season
        return teamGames

    teamLookup = pd.DataFrame(teams.get_teams())
    teamLookup = teamLookup.rename(columns={"id": "team_id", "nickname": "team_nickname"}).drop(columns='year_founded')

    conexion = sqlite3.connect("NBA_DATA.db")
    teamLookup.to_sql("Teams", conexion, if_exists="replace", index=False)

    scheduleFrame = pd.DataFrame()

    for season in seasons:
        for id in teamLookup['team_id']:
            time.sleep(10)
            teamGames = getRegularSeasonSchedule(season, id, seasonType)
            scheduleFrame = pd.concat([scheduleFrame, teamGames], ignore_index=True)

    scheduleFrame['GAME_DATE'] = pd.to_datetime(scheduleFrame['MATCHUP'].map(getGameDate))
    scheduleFrame['HOME_TEAM_NICKNAME'] = scheduleFrame['MATCHUP'].map(getHomeTeam)
    scheduleFrame['HOME_TEAM_ID'] = scheduleFrame['HOME_TEAM_NICKNAME'].map(getTeamIDFromNickname)
    scheduleFrame['AWAY_TEAM_NICKNAME'] = scheduleFrame['MATCHUP'].map(getAwayTeam)
    scheduleFrame['AWAY_TEAM_ID'] = scheduleFrame['AWAY_TEAM_NICKNAME'].map(getTeamIDFromNickname)
    scheduleFrame = scheduleFrame.drop_duplicates()  # There's a row for both teams, only need 1
    scheduleFrame = scheduleFrame.reset_index(drop=True)

    return scheduleFrame

In [5]:
# Get Single Game aggregation columns

def getSingleGameMetrics(gameID,homeTeamID,awayTeamID,awayTeamNickname,seasonYear,gameDate):

    @retry
    def getGameStats(teamID,gameID,seasonYear):
        #season = str(seasonYear) + "-" + str(seasonYear+1)[-2:]
        gameStats = cumestatsteam.CumeStatsTeam(game_ids=gameID,league_id ="00",
                                               season=seasonYear,season_type_all_star="Regular Season",
                                               team_id = teamID).get_normalized_json()

        gameStats = pd.DataFrame(json.loads(gameStats)['TotalTeamStats'])

        return gameStats

    data = getGameStats(homeTeamID,gameID,seasonYear)
    data.at[1,'NICKNAME'] = awayTeamNickname.strip()
    data.at[1,'TEAM_ID'] = awayTeamID
    data.at[1,'OFFENSIVE_EFFICIENCY'] = (data.at[1,'FG'] + data.at[1,'AST'])/(data.at[1,'FGA'] - data.at[1,'OFF_REB'] + data.at[1,'AST'] + data.at[1,'TOTAL_TURNOVERS'])
    data.at[1,'SCORING_MARGIN'] = data.at[1,'PTS'] - data.at[0,'PTS']

    data.at[0,'OFFENSIVE_EFFICIENCY'] = (data.at[0,'FG'] + data.at[0,'AST'])/(data.at[0,'FGA'] - data.at[0,'OFF_REB'] + data.at[0,'AST'] + data.at[0,'TOTAL_TURNOVERS'])
    data.at[0,'SCORING_MARGIN'] = data.at[0,'PTS'] - data.at[1,'PTS']

    data['SEASON'] = seasonYear
    data['GAME_DATE'] = gameDate
    data['GAME_ID'] = gameID

    return data

In [6]:
def getGameLogs(gameLogs,scheduleFrame):
    
    # Functions to prepare additional columns after gameLogs table loads
    def getHomeAwayFlag(gameDF):
        gameDF['HOME_FLAG'] = np.where((gameDF['W_HOME']==1) | (gameDF['L_HOME']==1),1,0)
        gameDF['AWAY_FLAG'] = np.where((gameDF['W_ROAD']==1) | (gameDF['L_ROAD']==1),1,0)
        #return gameDF 

    def getTotalWinPctg(gameDF):
        gameDF['TOTAL_GAMES_PLAYED'] = gameDF.groupby(['TEAM_ID','SEASON'])['GAME_DATE'].rank(ascending=True)
        gameDF['TOTAL_WINS'] = gameDF.sort_values(by='GAME_DATE').groupby(['TEAM_ID','SEASON'])['W'].cumsum()
        gameDF['TOTAL_WIN_PCTG'] = gameDF['TOTAL_WINS']/gameDF['TOTAL_GAMES_PLAYED']
        return gameDF.drop(['TOTAL_GAMES_PLAYED','TOTAL_WINS'],axis=1)

    def getHomeWinPctg(gameDF):
        gameDF['HOME_GAMES_PLAYED'] = gameDF.sort_values(by='GAME_DATE').groupby(['TEAM_ID','SEASON'])['HOME_FLAG'].cumsum()
        gameDF['HOME_WINS'] = gameDF.sort_values(by='GAME_DATE').groupby(['TEAM_ID','SEASON'])['W_HOME'].cumsum()
        gameDF['HOME_WIN_PCTG'] = gameDF['HOME_WINS']/gameDF['HOME_GAMES_PLAYED']
        return gameDF.drop(['HOME_GAMES_PLAYED','HOME_WINS'],axis=1)

    def getAwayWinPctg(gameDF):
        gameDF['AWAY_GAMES_PLAYED'] = gameDF.sort_values(by='GAME_DATE').groupby(['TEAM_ID','SEASON'])['AWAY_FLAG'].cumsum()
        gameDF['AWAY_WINS'] = gameDF.sort_values(by='GAME_DATE').groupby(['TEAM_ID','SEASON'])['W_ROAD'].cumsum()
        gameDF['AWAY_WIN_PCTG'] = gameDF['AWAY_WINS']/gameDF['AWAY_GAMES_PLAYED']
        return gameDF.drop(['AWAY_GAMES_PLAYED','AWAY_WINS'],axis=1)

    def getRollingOE(gameDF):
        gameDF['ROLLING_OE'] = gameDF.sort_values(by='GAME_DATE').groupby(['TEAM_ID','SEASON'])['OFFENSIVE_EFFICIENCY'].transform(lambda x: x.rolling(3, 1).mean())

    def getRollingScoringMargin(gameDF):
        gameDF['ROLLING_SCORING_MARGIN'] = gameDF.sort_values(by='GAME_DATE').groupby(['TEAM_ID','SEASON'])['SCORING_MARGIN'].transform(lambda x: x.rolling(3, 1).mean())

    def getRestDays(gameDF):
        gameDF['LAST_GAME_DATE'] = gameDF.sort_values(by='GAME_DATE').groupby(['TEAM_ID','SEASON'])['GAME_DATE'].shift(1)
        gameDF['NUM_REST_DAYS'] = (gameDF['GAME_DATE'] - gameDF['LAST_GAME_DATE'])/np.timedelta64(1,'D') 
        return gameDF.drop('LAST_GAME_DATE',axis=1)
    
    start = time.perf_counter_ns()

    i = int(len(gameLogs)/2) #Can use a previously completed gameLog datasetn

    while i<len(scheduleFrame):


        time.sleep(10)
        gameLogs = pd.concat([gameLogs, getSingleGameMetrics(scheduleFrame.at[i,'GAME_ID'],scheduleFrame.at[i,'HOME_TEAM_ID'],
                         scheduleFrame.at[i,'AWAY_TEAM_ID'],scheduleFrame.at[i,'AWAY_TEAM_NICKNAME'],
                         scheduleFrame.at[i,'SEASON'],scheduleFrame.at[i,'GAME_DATE'])])
        
        gameLogs = gameLogs.reset_index(drop=True)

        end = time.perf_counter_ns()

        if i%100 == 0:
            mins = ((end-start)/1e9)/60
            print(f"{i},{int(mins)}")

        i+=1
        
    # Get Table Level Aggregation Columns
    getHomeAwayFlag(gameLogs)
    gameLogs = getHomeWinPctg(gameLogs)
    gameLogs = getAwayWinPctg(gameLogs)
    gameLogs = getTotalWinPctg(gameLogs)
    getRollingScoringMargin(gameLogs)
    getRollingOE(gameLogs)
    gameLogs = getRestDays(gameLogs)

    return gameLogs.reset_index(drop=True)

In [9]:
#Get ScheduleFrame

seasons = [2021]
seasonType = 'Regular Season'

start = time.perf_counter_ns()
scheduleFrame = getSeasonScheduleFrame(seasons,seasonType)
scheduleFrame.to_sql("Games", conexion, if_exists="replace", index=False)
end = time.perf_counter_ns()

secs = (end-start)/1e9
mins = secs/60
print(f"scheduleFrame takes: {int(mins)}:{int(secs)%60}")

start = time.perf_counter_ns()
gameLogs = pd.DataFrame()
gameLogs = getGameLogs(gameLogs,scheduleFrame)
gameLogs.to_csv('gameLogs.csv')
gameLogs.to_sql("game_stats", conexion, if_exists="replace", index=False)

end = time.perf_counter_ns()

secs = (end-start)/1e9
mins = secs/60
print(f"GameLogs takes: {int(mins)}:{int(secs)%60}")

scheduleFrame takes: 5:11
0 0.17327623333333333
HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)
100 18.936819113333335
200 36.457881801666666
300 54.11950126166666
400 71.80678017
500 89.36442764333333
600 106.94538046166667
700 124.65639870333332
HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)
800 143.444021645
900 161.07449660333333
1000 178.76682765333334
1100 196.61668648833333
1200 214.38076823333333
GameLogs takes: 219:32


In [14]:
len(scheduleFrame.GAME_ID.unique())

1230

In [11]:
query = "SELECT * FROM game_stats"
gameLogs = pd.read_sql_query(query, conexion)
gameLogs = pd.DataFrame(gameLogs)
gameLogs

,CITY,NICKNAME,TEAM_ID,W,L,W_HOME,L_HOME,W_ROAD,L_ROAD,TEAM_TURNOVERS,...,GAME_DATE,GAME_ID,HOME_FLAG,AWAY_FLAG,HOME_WIN_PCTG,AWAY_WIN_PCTG,TOTAL_WIN_PCTG,ROLLING_SCORING_MARGIN,ROLLING_OE,NUM_REST_DAYS
0,Houston,Rockets,1610612745,0,1,0,1,0,0,3,...,2022-04-10 00:00:00,0022101221,1,0,0.268293,0.219512,0.243902,-10.333333,0.555047,2.0
1,OPPONENTS,Hawks,1610612737,1,0,0,0,1,0,1,...,2022-04-10 00:00:00,0022101221,0,1,0.658537,0.390244,0.524390,9.000000,0.593025,2.0
2,Miami,Heat,1610612748,1,0,1,0,0,0,0,...,2022-04-08 00:00:00,0022101207,1,0,0.707317,0.600000,0.654321,12.666667,0.604930,3.0
3,OPPONENTS,Hawks,1610612737,0,1,0,0,0,1,1,...,2022-04-08 00:00:00,0022101207,0,1,0.658537,0.375000,0.518519,0.333333,0.582568,2.0
4,Atlanta,Hawks,1610612737,1,0,1,0,0,0,0,...,2022-04-06 00:00:00,0022101192,1,0,0.658537,0.384615,0.525000,4.000000,0.576985,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2455,OPPONENTS,Pistons,1610612765,1,0,0,0,1,0,1,...,2022-02-27 00:00:00,0022100915,0,1,0.300000,0.193548,0.245902,-1.666667,0.555499,1.0
2456,Detroit,Pistons,1610612765,0,1,0,1,0,0,0,...,2022-02-11 00:00:00,0022100838,1,0,0.285714,0.142857,0.214286,-25.666667,0.545486,1.0
2457,OPPONENTS,Hornets,1610612766,1,0,0,0,1,0,0,...,2022-02-11 00:00:00,0022100838,0,1,0.538462,0.483871,0.508772,-1.666667,0.584475,2.0
2458,Charlotte,Hornets,1610612766,1,0,1,0,0,0,0,...,2022-01-05 00:00:00,0022100567,1,0,0.666667,0.416667,0.512821,-2.666667,0.603931,2.0


In [15]:
first_team_id = 1610612745

# Realizar una consulta para obtener toda la información correspondiente de la tabla 'Teams'
query = f"SELECT * FROM Teams WHERE team_id = {first_team_id}"
team_info = pd.read_sql_query(query, conexion)

# Mostrar el DataFrame con la información del equipo
print(team_info)

      team_id        full_name abbreviation team_nickname     city  state
0  1610612745  Houston Rockets          HOU       Rockets  Houston  Texas


In [17]:
teams =  pd.read_sql_query("select * from games", conexion)
teams = pd.DataFrame(teams)
teams

,MATCHUP,GAME_ID,SEASON,GAME_DATE,HOME_TEAM_NICKNAME,HOME_TEAM_ID,AWAY_TEAM_NICKNAME,AWAY_TEAM_ID
0,04/10/2022 Hawks at Rockets,0022101221,2021-22,2022-04-10 00:00:00,Rockets,1610612745,Hawks,1610612737
1,04/08/2022 Hawks at Heat,0022101207,2021-22,2022-04-08 00:00:00,Heat,1610612748,Hawks,1610612737
2,04/06/2022 Wizards at Hawks,0022101192,2021-22,2022-04-06 00:00:00,Hawks,1610612737,Wizards,1610612764
3,04/05/2022 Hawks at Raptors,0022101182,2021-22,2022-04-05 00:00:00,Raptors,1610612761,Hawks,1610612737
4,04/02/2022 Nets at Hawks,0022101163,2021-22,2022-04-02 00:00:00,Hawks,1610612737,Nets,1610612751
...,...,...,...,...,...,...,...,...
1225,11/22/2021 Hornets at Wizards,0022100254,2021-22,2021-11-22 00:00:00,Wizards,1610612764,Hornets,1610612766
1226,11/17/2021 Wizards at Hornets,0022100213,2021-22,2021-11-17 00:00:00,Hornets,1610612766,Wizards,1610612764
1227,02/27/2022 Pistons at Hornets,0022100915,2021-22,2022-02-27 00:00:00,Hornets,1610612766,Pistons,1610612765
1228,02/11/2022 Hornets at Pistons,0022100838,2021-22,2022-02-11 00:00:00,Pistons,1610612765,Hornets,1610612766
